In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
# import soundfile as sf
import librosa
import torch

print('load model and tokenizer')

device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
model = model.to(device)

print('load audio')
audio, rate = librosa.load("output.wav", sr = 16000)

print('tokenize')
input_values = tokenizer(audio, return_tensors = "pt").input_values.to(device)

print('prediction')
logits = model(input_values).logits

print('take argmax and decode')
prediction = torch.argmax(logits, dim = -1)
transcription = tokenizer.batch_decode(prediction)[0]

load model and tokenizer


c:\Users\clara\Desktop\greenfield_overall\asr\asr-venv\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:417: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


load audio
tokenize
prediction
take argmax and decode


In [4]:
import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print( "Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Microphone Array (Realtek(R) Au


In [5]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [5]:
print(transcription)
#open text file
text_file = open("cna_pred.txt", "w")
 
#write string to file
text_file.write(transcription)
 
#close file
text_file.close()

ONLY DELLY AND HI PAY OUR RAMPING A FREIGT TALK WHICH CULD BRING AS SEVEN QI IE BILLYA DOLLARS SAMYTE AT TA BACT RETED INDIA BY THE AN OF A YEAR BID CID BECOME ANOTHER FA FOINT  BETWEEN YOU DALLY AND BA JAKE A RELATIONS REMAIN HEN A TRUE A FILL DEALING WITH A BORDE DISPUTE FAN LASTDA AND HEMALAYAN REGION A MODER ME APON YOUR HAV MORE MEDE REPORT SOR GEST INDIA AND FIVARN ARE AN TALKS OVER AN A GREEMEN THAT COULD HELT OE DRES SNE ALLY TO TET IP SHIP SHORT AF FOR A BROOMBOR REFORTH OFFICIALS IN OUJALLY AND SIE TAY A MET A GEEANT WY TO THE CAFFEDEEL THAT WOULD HAL GRIN AT SHIP FLANT TO INDIA EXPECTEDLY WORTH TAVEN CON FIVE FBIDION DOLLARS IT WILL HEL  TE LIE SHIP WOR EVERYING FROM FIVE TY TAVIVEWIT TO WE THAT TRACART IN DHEAS O REBOTH OF THES SCALTIN FOFF ABLE LOCATION FOR THE PLANT THA GOVERMENT ES FOMM TACK BREAT TAT OF WE DUCTIONS AS WELL A HE C MI MIN TO PROVIDE FIFTY PE TANT O THE CAPITAL EXTAN YCANI BY TWENTY  TWENTY THREE CARRENTLY INDIA IN FORT ALL OF THET SANY CONDUCTORS BUT DEMA

In [16]:
def wer(ref, hyp ,debug=False):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
     
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
         
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
     
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 'Del':numDel}

In [17]:
with open('cna_GT.txt', 'r') as file:
    cna_GT = file.read().replace('\n', ' ')

with open('cna_pred.txt', 'r') as file:
    cna_pred = file.read().replace('\n', '')

WER = wer(cna_GT, cna_pred)

print(WER)

{'WER': 1.096, 'Cor': 0, 'Sub': 394, 'Ins': 38, 'Del': 0}
